## Import the Pickle

In [1]:
import pickle
from langdetect import detect
import re
import string
#from project_functions import *
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import sentiment_mod as s

In [2]:
!ls /home/jovyan/work/2_Semester

'API Tweeter.ipynb'   featuresets.pickle	      Text_Mining_MASTER
 backup		     'Machine Learning'		      tweets_df.csv
'Deep Learning'      'PLaying with tutorials.ipynb'   tweets_jan2019


In [3]:
pickle_tweets = open("/home/jovyan/work/2_Semester/tweets_jan2019", "rb")
example_tweets = pickle.load(pickle_tweets)
example_tweets = example_tweets[["datetime","text"]]
example_tweets.head()

,datetime,text
0,2019-01-01 00:00:00,"@ #1, Bitcoin with unit price of $3,742.7, m..."
1,2019-01-01 00:00:00,Learn it. 2018 Sees Bitcoin ’s Lowest Average...
2,2019-01-01 00:00:01,"仮想通貨の時価総額 $125,622,003,150 BTC 価格:$3734.04..."
3,2019-01-01 00:00:01,IAM Platform Curated Retweet: Via: https:// ...
4,2019-01-01 00:00:01,"Bitcoin - BTC Price: $3,742.70 Change in 1h: ..."


### Take a sample

### Filter on English Tweets

In [4]:
def filter_english(dataframe):
    blanco = "blanco"
    text_column = dataframe["text"]
    
    # Create a list saving all the languages of the tweets
    language_list =[]

    for i in text_column:

        try:
            language = detect(i)
            language_list.append(language)
        except:
            language_list.append(blanco)    
    
    dataframe["Language"] = language_list
    
    return dataframe.loc[dataframe['Language'] == "en"]


In [5]:
example_tweets = filter_english(example_tweets)

### Clean Tweets

In [9]:
def removal_function(dataframe):
    new_text = []
    text_column = dataframe["text"]
    for i in text_column:
        y = i

        y = re.sub(r"@[A-Z-a-z-0-9_.]+","", y) #remove users with@
        y = y.replace("\n"," ") # remove enters
        y= re.sub(r"http\S+","",y) # removes links
        y= re.sub("\s+"," ",y)  #removes more one spaces
        y= re.sub(r"&(amp;)", "&", y) # removes and in html format
        y = re.sub(r"[0-9]","",y) #remove numbers
        y=re.sub(r"(.+?)\1+",r"\1",y) #remove repeted letters
        y= re.sub("\s+"," ",y) #remove more one space

        i = y
        new_text.append(i)
        
    dataframe["text"] = new_text
    return dataframe

In [10]:
example_tweets = removal_function(example_tweets)

### Tokenize, Remove Stopwords, Lemmatize, Stemmatize

In [12]:
def tokenize_dataframe(dataframe):
    text_column = dataframe["text"]
    new_text = []
    
    for i in text_column:
        i = i.lower()
        i = RegexpTokenizer(r'\w+').tokenize(i)
        new_text.append(i)
        
    text_column = new_text
    dataframe["text"] = text_column
    
    return dataframe

In [13]:
example_tweets = tokenize_dataframe(example_tweets)

In [15]:
def remove_stopwords_dataframe(dataframe):
    text_column = dataframe["text"]
    new_words = []
    
    for i in text_column:
        stop_words = set(stopwords.words("english"))
        stop_text = [j for j in i if not j in stop_words]
        new_words.append(stop_text)
    
    text_column = new_words
    dataframe["text"] = text_column
    
    return dataframe

In [16]:
example_tweets = remove_stopwords_dataframe(example_tweets)

In [ ]:
def lemmatize_dataframe(dataframe):
    wordnet = WordNetLemmatizer()
    text_column = dataframe["text"]
    new_words = []
    
    for i in text_column:
        lemma = [wordnet.lemmatize(token) for token in i]
        new_words.append(lemma)
        
    text_column = new_words
    dataframe["text"] = text_column
    
    return dataframe
        

In [ ]:
def stemmatize_dataframe(dataframe):
    stemmer = nltk.SnowballStemmer("english")
    text_column = dataframe["text"]
    new_words= []
    
    for i in text_column:
        stemmed = [stemmer.stem(token) for token in i]
        new_words.append(stemmed)
    
    text_column = new_words
    dataframe["text"] = text_column
    
    return dataframe

In [ ]:
def untokenize_dataframe(dataframe):
    text_column = dataframe["text"]
    new_text = []
    
    for i in text_column:
        i = " ".join(i)
        new_text.append(i)
        
    text_column = new_text
    dataframe["text"] = text_column
    
    return dataframe

In [ ]:
example_tweets = untokenize_dataframe(example_tweets)

### Sentiment Using TextBlob

In [ ]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
def add_sentiment_textblob(dataframe):
    text_column = dataframe["text"]
    sentiment_textblob_list = []

    for i in text_column:
        sentiment_value, polarity = s.sentiment_textblob(i)#sentiment_textblob(i)
        sentiment_textblob_list.append(sentiment_value)

    dataframe["sentiment_textblob"] = sentiment_textblob_list
    
    return dataframe

In [ ]:
example_tweets = add_sentiment_textblob(example_tweets)

### Sentiment Using NLTK

In [ ]:
def add_sentiment_nltk(dataframe):
    text_column = dataframe["text"]
    sentiment_nltk_list = []

    for i in text_column:
        sentiment_value, polarity = s.sentiment_nltk(i)#sentiment_textblob(i)
        sentiment_nltk_list.append(sentiment_value)

    dataframe["sentiment_nltk"] = sentiment_nltk_list
    
    return dataframe

In [ ]:
example_tweets = add_sentiment_nltk(example_tweets)

### Sentiment own trained models

In [ ]:
def add_sentiment_own(dataframe):
    text_column = dataframe["text"]
    sentiment_own_list = []

    for i in text_column:
        sentiment_value, polarity = s.sentiment(i)#sentiment_textblob(i)
        if polarity < 0.75:
            sentiment_own_list.append("neutral")
        else:
            sentiment_own_list.append(sentiment_value)

    dataframe["sentiment_own_classifiers"] = sentiment_own_list
    
    return dataframe

In [ ]:
example_tweets = add_sentiment_own(example_tweets)

### Graph sentiment & BTC time series

In [ ]:
import copy
example_tweets_graph = copy.deepcopy(example_tweets)